In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import pandas as pd
from utility import *

In [4]:
X, y = load_data()
enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

In [5]:
def time_step(data, label, step=10):
    x,y = [],[]
    for i in range(data.shape[0]-(step+1)):
        x.append(data[i:i+step])
        y.append(label[i+step+1])
    return np.array(x),np.array(y)

In [6]:
def create_dataset(team_name):
    ht = data.loc[data['HomeTeam']==team_name]
    at = data.loc[data['AwayTeam']==team_name]
    team_data = pd.concat([ht,at])
    team_data_label = team_data['FTR']
    team_data_featured = pd.get_dummies(team_data[features])
    return team_data_featured,team_data_label

In [7]:
team_name = 'Chelsea'
data_f, data_l = create_dataset(team_name)
data_f = SimpleImputer().fit_transform(data_f)
x_ars, y_ars = time_step(data_f, data_l.to_numpy())
print(x_ars.shape)
print(y_ars.shape)

NameError: name 'data' is not defined

In [ ]:
y_ars = enc.fit_transform(y_ars.reshape(-1,1))

In [ ]:
x_train_lstm, x_test_lstm, y_train_lstm, y_test_lstm = train_test_split(x_ars, y_ars, shuffle=False, test_size=test_size)

In [ ]:
Tx = x_train_lstm.shape[1] #Time steps
Ty = y_train_lstm.shape[1] #Time Steps

# LSTM

In [ ]:
import os

model = None
model_path = './models/lstm/'

# Check se il modello è già salvato
if not os.path.exists(model_path) or os.listdir(model_path):
    os.mkdir("./" + model_path.split("/")[1])
    os.mkdir(model_path)


else:
    model = k.models.load_model(model_path)


if model == None:
    model = k.models.Sequential(
        [
            k.layers.LSTM(64, return_sequences=True),
            k.layers.Dropout(0.4),
            k.layers.Dense(1000, activation="relu"),
            k.layers.Dropout(0.3),
            k.layers.Dense(250, activation="relu"),
            k.layers.Dropout(0.2),
            k.layers.Dense(3, activation="softmax")
        ]
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=["accuracy"]
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    model.fit(x_train_lstm, y_train_lstm, epochs=10, callbacks=[callback])
    model.save('./models/lstm')

Epoch 1/10


ValueError: in user code:

    File "/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_lo

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

## 2 LSTM concatenate + dense e dropout in sequenza

In [ ]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True, activation="relu")(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True, activation="relu")(inputs)
concateneted = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concateneted)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)
report(model, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

### Fit
loss: 0.9373 - accuracy: 0.5609
### Evaluate
loss: 0.9192 - accuracy: 0.5930
### Classification report

| | precision | recall | f1-score | support |
| - | ----------- | ------ | -------- | ------- |
| A | 0.609 | 0.614 | 0.612 | 345 |
| D | 0.321 | 0.201 | 0.247 | 254 |
| H | 0.652 | 0.763 | 0.704 | 541 |
| accuracy | |  | 0.593 | 1140 |
| macro avg | 0.527 | 0.526 | 0.521 | 1140 |
| weighted avg | 0.565 | 0.593 | 0.574 | 1140 |

## LSTM in parallelo

In [ ]:
inputs = k.layers.Input(shape=(10,94))
x = []
for t in range(Tx):
    module = k.layers.LSTM((t+1)*10, return_sequences=True)(inputs)
    module = k.layers.Dropout(0.7)(module)
    x.append(module)
x = k.layers.Concatenate()(x)

out = k.layers.Dense(1000, activation="relu")(x)
out = k.layers.Dropout(0.7)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dense(3, activation="softmax")(out)

txLstm = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
txLstm.summary()

In [ ]:
txLstm.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

txLstm.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback], batch_size=16)

In [ ]:
txLstm.evaluate(y_test_lstm, y_test_lstm)
report(txLstm, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

# GRU

In [ ]:
gru = k.models.Sequential([
    k.layers.GRU(100, return_sequences=True, activation="relu"),
    k.layers.Dropout(0.4),
    k.layers.Dense(1000, activation="relu"),
    k.layers.Dropout(0.3),
    k.layers.Dense(250, activation="relu"),
    k.layers.Dropout(0.2),
    k.layers.Dense(3, activation="softmax")
])

In [ ]:
gru(x_train_lstm)
gru.summary()

In [ ]:
gru.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

gru.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
gru.evaluate(x_test_lstm, y_test_lstm)
report(gru, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

# Time distributed

In [ ]:
model = k.models.Sequential(
    [
        k.layers.GRU(128, dropout=0.5, return_sequences=True),
        k.layers.Flatten(),
        k.layers.Dense(40, activation="relu"),
        k.layers.Dropout(0.4),
        k.layers.Dense(20, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(10, activation="relu"),
        k.layers.Dense(3, activation="softmax")
    ]
)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)
model(x_train_lstm)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (296, 10, 128)            86016     
                                                                 
 flatten_1 (Flatten)         (296, 1280)               0         
                                                                 
 dense_9 (Dense)             (296, 40)                 51240     
                                                                 
 dropout_7 (Dropout)         (296, 40)                 0         
                                                                 
 dense_10 (Dense)            (296, 20)                 820       
                                                                 
 dropout_8 (Dropout)         (296, 20)                 0         
                                                                 
 dense_11 (Dense)            (296, 10)                

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(x_train_lstm, y_train_lstm, epochs=1000, callbacks=[callback])

Epoch 1/1000


2022-12-14 18:58:10.696297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-14 18:58:10.845232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-14 18:58:11.326873: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 3s 107ms/step - loss: 1.1161 - accuracy: 0.4865
Epoch 2/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0888 - accuracy: 0.4662
Epoch 3/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0973 - accuracy: 0.4764
Epoch 4/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0625 - accuracy: 0.5169
Epoch 5/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0245 - accuracy: 0.5304
Epoch 6/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0262 - accuracy: 0.5236
Epoch 7/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0277 - accuracy: 0.5203
Epoch 8/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0053 - accuracy: 0.5304
Epoch 9/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0400 - accuracy: 0.5507
Epoch 10/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0489 - accuracy: 0.5304
E

In [ ]:
model.evaluate(x_test_lstm, y_test_lstm)

report(model, x_train_lstm, y_train_lstm, "train", enc)
report(model, x_test_lstm, y_test_lstm, "test", enc)

10/10 [==============================] - 0s 4ms/step
TRAIN REPORT
              precision    recall  f1-score   support

           A      0.447     0.551     0.494        69
           D      0.000     0.000     0.000        64
           H      0.649     0.840     0.733       163

    accuracy                          0.591       296
   macro avg      0.365     0.464     0.409       296
weighted avg      0.462     0.591     0.518       296

--------------------------------------------------
4/4 [==============================] - 0s 6ms/step
TEST REPORT
              precision    recall  f1-score   support

           A      0.556     1.000     0.714        60
           D      0.000     0.000     0.000        21
           H      0.000     0.000     0.000        30

    accuracy                          0.541       111
   macro avg      0.185     0.333     0.238       111
weighted avg      0.300     0.541     0.386       111

--------------------------------------------------


/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/mdl_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  